<a href="https://colab.research.google.com/github/merrouchi37/Covid-19_detection/blob/master/Covid_19_Training_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("/content/drive/My Drive/app")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, VGG16, VGG19, ResNet50
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,RMSprop
import matplotlib.pyplot as plt
import numpy as np
#import argparse
import cv2
import os
import shutil

In [ ]:
# Load data for training and validation
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')
data_val = np.load('data_val.npy')
labels_val = np.load('labels_val.npy')

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=15
	)

In [ ]:
# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False, 
	input_tensor=Input(shape=(224, 224, 3)))

# For the other 3 models, the corresponding line of code is executed each time as follows:
#baseModel = VGG19(weights="imagenet", include_top=False, 
#	input_tensor=Input(shape=(224, 224, 3)))
#baseModel = InceptionV3(weights="imagenet", include_top=False, 
#	input_tensor=Input(shape=(224, 224, 3)))
#baseModel = ResNet50(weights="imagenet", include_top=False, 
#	input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
baseModel.summary()

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
x = baseModel.output
x = AveragePooling2D(pool_size=(2, 2))(x)
x = Flatten(name="flatten")(x)
x = Dense(192, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(3, activation="softmax")(x)

In [ ]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=x)

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process

for layer in baseModel.layers[:7]:     # for VGG16
#for layer in baseModel.layers[:7]:    # for VGG19
#for layer in baseModel.layers[:102]:  # for InceptionV3
#for layer in baseModel.layers[:19]:   # for ResNet50
	layer.trainable = False

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 15
BS_tr = 32
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])

In [ ]:
mcp_save = ModelCheckpoint('covid19_models/best_vgg16.h5', save_best_only=True, monitor='val_acc', mode='max')
#mcp_save = ModelCheckpoint('covid19_models/best_vgg19.h5', save_best_only=True, monitor='val_acc', mode='max')
#mcp_save = ModelCheckpoint('covid19_models/best_InceptionV3.h5', save_best_only=True, monitor='val_acc', mode='max')
#mcp_save = ModelCheckpoint('covid19_models/best_resNet50.h5', save_best_only=True, monitor='val_acc', mode='max')
callbacks = [ mcp_save]

In [ ]:
# train the head of the network
print("[INFO] training head...")
EPOCHS=15
BS_tr=32
BS_val = 16
H = model.fit_generator(
	trainAug.flow(data_train, labels_train, batch_size=BS_tr),
	steps_per_epoch=len(data_train) // BS_tr ,
	validation_data=(data_val, labels_val),
	validation_steps=len(data_val) // BS_val ,
	epochs=EPOCHS, callbacks=callbacks)